
## cn_stock/bars/daily

### Overwrite

In [3]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    with open("finance.duckdb.sql") as f:
        # conn.execute("drop table bars_daily_000001_SZ;")
        sql = f.read().format(tablename='000001_SZ', csv_path='datasources/000001.SZ')
        conn.execute(sql)
    conn.table('bars_daily_000001_SZ').show()

┌─────────────────────┬──────────┬───────┬───────┬───────┬───┬───────────────┬─────────┬─────────┬─────────┬─────────┐
│         dt          │   name   │ _open │ _high │ _low  │ … │ lb_down_count │  close  │  open   │  high   │   low   │
│     timestamp_s     │ varchar  │ float │ float │ float │   │     uint8     │  float  │  float  │  float  │  float  │
├─────────────────────┼──────────┼───────┼───────┼───────┼───┼───────────────┼─────────┼─────────┼─────────┼─────────┤
│ 1991-01-03 00:00:00 │ 深发展Ａ │  66.4 │  66.4 │  66.4 │ … │             0 │ 7.07077 │ 7.07077 │ 7.07077 │ 7.07077 │
│ 1991-01-04 00:00:00 │ 深发展Ａ │ 66.07 │ 66.07 │ 66.07 │ … │             0 │ 7.03562 │ 7.03562 │ 7.03562 │ 7.03562 │
│ 1991-01-07 00:00:00 │ 深发展Ａ │ 66.42 │ 66.42 │ 66.42 │ … │             0 │  7.0729 │  7.0729 │  7.0729 │  7.0729 │
│ 1991-01-09 00:00:00 │ 深发展Ａ │ 65.76 │ 65.76 │ 65.76 │ … │             0 │ 7.00261 │ 7.00261 │ 7.00261 │ 7.00261 │
│ 1991-01-10 00:00:00 │ 深发展Ａ │ 65.43 │ 65.43 │ 65.43 │ … │      

In [2]:
import duckdb

with duckdb.connect("datas/duckdb/finance_ticks_ctpfuture.db") as conn:
    with open("finance.duckdb.ticks.sql") as f:
        # conn.execute("drop table ticks_c;")
        sql = f.read().format(tablename='c', csv_path='datasources/c') # c 有3个G没有上传git了
        conn.execute(sql)
    conn.table('ticks_c').show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬────────────┬────────┬──────────┬───┬────────────┬────────────┬─────────────┬─────────────┐
│          dt          │ last_price │ volume │  amount  │ … │ bid_price1 │ ask_price1 │ bid_volume1 │ ask_volume1 │
│      timestamp       │   float    │ uint32 │  uint32  │   │   float    │   float    │   uint32    │   uint32    │
├──────────────────────┼────────────┼────────┼──────────┼───┼────────────┼────────────┼─────────────┼─────────────┤
│ 2017-01-04 09:00:0…  │     1521.0 │    985 │ 14988160 │ … │     1520.0 │     1521.0 │         143 │          15 │
│ 2017-01-04 09:00:0…  │     1521.0 │    384 │  5841550 │ … │     1520.0 │     1521.0 │         103 │          90 │
│ 2017-01-04 09:00:0…  │     1521.0 │     58 │   882110 │ … │     1520.0 │     1521.0 │         201 │          76 │
│ 2017-01-04 09:00:0…  │     1521.0 │     49 │   745150 │ … │     1520.0 │     1521.0 │         289 │          68 │
│ 2017-01-04 09:00:0…  │     1520.0 │    458 │  6963530 │ … │     1520.0

### Write

In [4]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    insert_sql = f"INSERT INTO bars_daily_000001_SZ SELECT dt,name,_open,_high,_low,_close,volume,amount,preclose,net_profit_ttm,cashflow_ttm,equity,asset,debt,debttoasset,net_profit_q,pe_ttm,pb,mkt_cap,mkt_cap_ashare,vip_buy_amt,vip_sell_amt,inst_buy_amt,inst_sell_amt,mid_buy_amt,mid_sell_amt,indi_buy_amt,indi_sell_amt,master_net_flow_in,master2_net_flow_in,vip_net_flow_in,mid_net_flow_in,inst_net_flow_in,indi_net_flow_in,total_sell_amt,total_buy_amt,net_flow_in,turnover,free_shares,total_shares,maxupordown,maxupordown_at_open,lb_up_count,lb_down_count,close,open,high,low FROM 'datasources/data_append.csv' where symbol='000001.SZ';"
    conn.execute(insert_sql)

### Delete

In [5]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    # 删除不存在的行不会报错
    insert_sql = f"DELETE FROM bars_daily_000001_SZ WHERE dt = '2024-09-02 00:00:00';"
    conn.execute(insert_sql)

### Read

In [4]:
import quantdata as qd

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/test_finance.db')
    # df = qd.duckdb_get_array(conn, "test_finance", 'bars_daily_000001_SZ').df()
    # df = qd.duckdb_get_array(conn, "test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open']).df()
    # df = qd.duckdb_get_array(conn, "test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'], filter="dt>'2024-08-29 00:00:00' and open > 10.08").df()
    # 最后一行(所有列)
    # df = qd.duckdb_get_array_last_rows(conn, "test_finance", 'bars_daily_000001_sz').df()
    # 最后一行(自定义列)
    # df = qd.duckdb_get_array_last_rows(conn, "test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open']).df()
    # 最后5行
    df = qd.duckdb_get_array_last_rows(conn, "test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'], N=5).df()
    # df = qd.duckdb_get_array_last_rows(conn, "test_finance", 'bars_daily_000001_SZ', N=1).df()
df

,dt,name,open
0,2024-08-26,平安银行,10.50
1,2024-08-27,平安银行,10.47
2,2024-08-28,平安银行,10.41
3,2024-08-29,平安银行,10.31
4,2024-08-30,平安银行,10.11


In [2]:
import quantdata as qd

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/test_finance.db')
    q = qd.duckdb_get_array(conn, "test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'])
    recarray = qd.duckdb_fetchrecarray(q)
recarray

rec.array([('1991-01-03T00:00:00', '',  7.07077),
           ('1991-01-04T00:00:00', '',  7.03562),
           ('1991-01-07T00:00:00', '',  7.0729 ), ...,
           ('2024-08-28T00:00:00', '', 10.41   ),
           ('2024-08-29T00:00:00', '', 10.31   ),
           ('2024-08-30T00:00:00', '', 10.11   )],
          dtype=[('dt', '<M8[s]'), ('name', '<U'), ('open', '<f4')])

### Benchmark

以下为在大数据(全部5000+只股票的数据，一共5G)测试性能

In [6]:
import quantdata as qd

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    %timeit qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ').fetchnumpy()
    %timeit  qd.duckdb_get_array(conn, "finance", 'bars_daily_000002_SZ', attrs=['dt', 'name', 'open']).fetchnumpy()
    %timeit  qd.duckdb_get_array(conn, "finance", 'bars_daily_000003_SZ', attrs=['dt', 'name', 'open'], filter="dt>'2024-08-29 00:00:00' and open > 10.08").fetchnumpy()
    # # 最后一行(所有列)
    %timeit  qd.duckdb_get_array_last_rows(conn, "finance", 'bars_daily_000005_SZ').fetchnumpy()
    # # 最后一行(自定义列)
    %timeit  qd.duckdb_get_array_last_rows(conn, "finance", 'bars_daily_000008_SZ', attrs=['dt', 'name', 'open']).fetchnumpy()
    # # 最后5行
    %timeit qd.duckdb_get_array_last_rows(conn, "finance", 'bars_daily_000010_SZ', attrs=['dt', 'name', 'open'], N=5).fetchnumpy()
    %timeit  qd.duckdb_get_array_last_rows(conn, "finance", 'bars_daily_000001_SZ', N=5).fetchnumpy()

5.12 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.13 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
586 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.68 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
909 µs ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
879 µs ± 30.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.06 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


以下为 duckdb_attach 函数测试，分为
- 本地文件
- 局域网内共享文件
- HTTP 文件

In [1]:
import quantdata as qd

def test_conn01():
    conn = qd.duckdb_connect()
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    qd.duckdb_close(conn)

test_conn01()

In [2]:
import quantdata as qd

def test_conn02():
    conn = qd.duckdb_connect()
    qd.duckdb_attach(conn, '//DESKTOP-6NI30V4/bt_data/duckdb/finance.db')
    qd.duckdb_close(conn)

test_conn02()

In [ ]:
import quantdata as qd

def test_conn03():
    conn = qd.duckdb_connect(extensions=('httpfs',))
    qd.duckdb_attach(conn, 'http://192.168.2.40:9001/finance.db')
    qd.duckdb_close(conn)

test_conn03()

以下为 fetchxxx() 函数测试

In [5]:
import quantdata as qd

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    %timeit qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ').fetchnumpy()

6.33 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
import quantdata as qd

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    %timeit qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ').fetchdf()

10.2 ms ± 654 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
import quantdata as qd

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    %timeit qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ').fetchall()

71.1 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
import quantdata as qd
import numpy as np

duckdbpytype_to_nptype = {
    "BIGINT": np.int64,
    # "BIT": ,
    # "BLOB": ,  # bytearray or bytes
    "BOOLEAN": "?",
    "DATE": "datetime64[D]",
    "DOUBLE": np.float64,
    "FLOAT": np.float32,
    # "HUGEINT": np.int128,
    # "UHUGEINT": np.uint128,
    "INTEGER": np.int32,
    # "INTERVAL":
    "SMALLINT": np.int16,
    # "SQLNULL": ,
    # "TIME": ,
    "TIMESTAMP": "datetime64[us]",  # microsecond
    "TIMESTAMP_MS": "datetime64[ms]",  # millisecond
    "TIMESTAMP_NS": "datetime64[ns]",
    "TIMESTAMP_S": "datetime64[s]",
    # "TIMESTAMP_TZ":
    # "TIME_TZ":
    "TINYINT": np.int8,
    "UBIGINT": np.uint64,
    "UINTEGER": np.uint32,
    "USMALLINT": np.uint16,
    "UTINYINT": np.uint8,
    # "UUID":
    "VARCHAR": "U",
}


def duckdb_fetchrecarray(q):
    dtypes = [
        (field, duckdbpytype_to_nptype[str(dtype)])
        for field, dtype in zip(q.columns, q.dtypes)
    ]
    return np.array(q.fetchall(), dtype=dtypes).view(np.recarray)

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    rec = duckdb_fetchrecarray(qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ'))
    %timeit duckdb_fetchrecarray(qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ'))
rec

108 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


rec.array([('1991-01-03T00:00:00', '', 66.4 , 66.4 , 66.4 , 66.4 ,     21200,    1410000, 67.41, 0.0000e+00, 0.00000e+00, 0.00000e+00, 0.000000e+00, 0.000000e+00, 0.     , 0.0000e+00, 0.    , 0.    , 4.50855987e+09, 2.46343987e+09,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,      0.  ,      0.  ,  0.  , 0.00057,    37099996,    67900000, 0, 0, 0, 0,  7.07077,  7.07077,  7.07077,  7.07077),
           ('1991-01-04T00:00:00', '', 66.07, 66.07, 66.07, 66.07,     16800,    1109000, 66.4 , 0.0000e+00, 0.00000e+00, 0.00000e+00, 0.000000e+00, 0.000000e+00, 0.     , 0.0000e+00, 0.    , 0.    , 4.48615322e+09, 2.45119693e+09,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,      0.  ,      0.  ,  0.  , 0.00045,    37100000,    67900000, 0, 0, 0, 0,  7.03562,  7.03562,  7.03562,  7.03562),
           ('1991-01-0

In [17]:
import quantdata as qd
import numpy as np


def duckdb_fetchrecarray(q):
    datas = q.fetchnumpy()
    dtypes = [(col, arr.dtype) for col, arr in datas.items()]
    return np.array(list(zip(*datas.values())), dtype=dtypes).view(np.recarray)

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    rec = duckdb_fetchrecarray(qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ'))
    %timeit duckdb_fetchrecarray(qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ'))
rec

67.5 ms ± 6.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


rec.array([('1991-01-03T00:00:00', '深发展Ａ', 66.4 , 66.4 , 66.4 , 66.4 ,     21200,    1410000, 67.41, 0.0000e+00, 0.00000e+00, 0.00000e+00, 0.000000e+00, 0.000000e+00, 0.     , 0.0000e+00, 0.    , 0.    , 4.50855987e+09, 2.46343987e+09,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,      0.  ,      0.  ,  0.  , 0.00057,    37099996,    67900000, 0, 0, 0, 0,  7.07077,  7.07077,  7.07077,  7.07077),
           ('1991-01-04T00:00:00', '深发展Ａ', 66.07, 66.07, 66.07, 66.07,     16800,    1109000, 66.4 , 0.0000e+00, 0.00000e+00, 0.00000e+00, 0.000000e+00, 0.000000e+00, 0.     , 0.0000e+00, 0.    , 0.    , 4.48615322e+09, 2.45119693e+09,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,      0.  ,      0.  ,  0.  , 0.00045,    37100000,    67900000, 0, 0, 0, 0,  7.03562,  7.03562,  7.03562,  7.03562),
           ('1

In [18]:
import quantdata as qd

def duckdb_fetchrecarray(q):
    return q.fetchdf().to_records(index=False)

with qd.duckdb_connect() as conn:
    qd.duckdb_attach(conn, 'datas/duckdb/finance.db')
    rec = duckdb_fetchrecarray(qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ'))
    %timeit duckdb_fetchrecarray(qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ'))
rec

37 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


rec.array([('1991-01-03T00:00:00', '深发展Ａ', 66.4 , 66.4 , 66.4 , 66.4 ,     21200,    1410000, 67.41, 0.0000e+00, 0.00000e+00, 0.00000e+00, 0.000000e+00, 0.000000e+00, 0.     , 0.0000e+00, 0.    , 0.    , 4.50855987e+09, 2.46343987e+09,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,      0.  ,      0.  ,  0.  , 0.00057,    37099996,    67900000, 0, 0, 0, 0,  7.07077,  7.07077,  7.07077,  7.07077),
           ('1991-01-04T00:00:00', '深发展Ａ', 66.07, 66.07, 66.07, 66.07,     16800,    1109000, 66.4 , 0.0000e+00, 0.00000e+00, 0.00000e+00, 0.000000e+00, 0.000000e+00, 0.     , 0.0000e+00, 0.    , 0.    , 4.48615322e+09, 2.45119693e+09,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ,      0.  ,      0.  ,  0.  , 0.00045,    37100000,    67900000, 0, 0, 0, 0,  7.03562,  7.03562,  7.03562,  7.03562),
           ('1